In [1]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import random
import torch
import re
import json

In [5]:
# Veri setlerini yükle
datasets = {
    "arc": load_dataset("malhajar/arc-tr-v0.2", split="test"),
    "hellaswag": load_dataset("malhajar/hellaswag-tr", split="validation"),
    "gsm8k": load_dataset( "malhajar/gsm8k-tr", "main", split="test")
}

README.md:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

hellaswag_train_14740.jsonl:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

hellaswag_train_39904.jsonl:   0%|          | 0.00/55.5M [00:00<?, ?B/s]

hellaswag_validation_10041.jsonl:   0%|          | 0.00/18.1M [00:00<?, ?B/s]

hellaswag_test_10002.jsonl:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

main/train-7472.jsonl:   0%|          | 0.00/5.46M [00:00<?, ?B/s]

main/test-1318.jsonl:   0%|          | 0.00/987k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7472 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1318 [00:00<?, ? examples/s]

In [24]:
dataset = load_dataset("malhajar/arc-tr-v0.2", split="test")
dataset_hellas = load_dataset("malhajar/hellaswag-tr", split="validation")
dataset_gsm = load_dataset( "malhajar/gsm8k-tr", "main", split="test")

In [6]:
# Model ve tokenizer'ı yükle
model_path = "/content/drive/MyDrive/Colab Notebooks/CNL/cache/cosmos_dpo"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda", torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
# Prompt formatlama fonksiyonu (ARC için)
def format_prompt_arc(example, shots):
    prompt = ""
    for shot in shots:
        if 'answerKey_letter' in shot: # Shot'ta harfsel cevap varsa
            shot_answer = shot['answerKey_letter']
        else:
            try:
                shot_answer_index = int(shot['answerKey']) - 1
                shot_answer = chr(65 + shot_answer_index)
            except ValueError: # Shot'ta harfsel cevap varsa
                shot_answer = shot['answerKey']
            except KeyError:
                shot_answer = ""
        prompt += f"Soru: {shot['question']}\n"
        for i, choice in enumerate(shot['choices']):
            prompt += f"{chr(65 + i)}. {choice}\n"
        prompt += f"Cevap: {shot_answer}\n\n"
    prompt += f"Soru: {example['question']}\n"
    for i, choice in enumerate(example['choices']):
        prompt += f"{chr(65 + i)}. {choice}\n"
    prompt += "Cevap:"
    return prompt

In [30]:
# Prompt formatlama fonksiyonu (Hellaswag için) (Düzeltilmiş)
def format_prompt_hellaswag(example, shots):
    prompt = ""
    for shot in shots:
        label_index = int(shot['label']) # Kesin çözüm: label'ı int'e çevir
        prompt += f"{shot['ctx']} {shot['endings'][label_index]}\n\n"
    prompt += f"{example['ctx']} "
    return prompt

In [39]:
# Prompt formatlama fonksiyonu (GMSK8K için)
def format_prompt_gsm(example, shots):
    prompt = ""
    for shot in shots:
        prompt += f"Soru: {shot['question']}\nCevap: {shot['answer']}\n\n"
    prompt += f"Soru: {example['question']}\nCevap: "
    return prompt

In [16]:
# Tahmin fonksiyonları
def get_prediction_arc(example, shots):
    prompt = format_prompt_arc(example, shots)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=1, pad_token_id=tokenizer.eos_token_id)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Cevap:")[-1].strip()
    return prediction

In [31]:
# Tahmin fonksiyonu (Hellaswag için)
def get_prediction_hellaswag(example, shots):
    prompt = format_prompt_hellaswag(example, shots)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split()[-1].strip()
    return prediction

In [38]:
# Tahmin fonksiyonu (GMSK8K için)
def get_prediction_gsm(example, shots):
    prompt = format_prompt_gsm(example, shots)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id) # max_new_tokens'ı artırdık
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Cevap:")[-1].strip() # Cevap kısmından sonrasını al
    return prediction

In [40]:
# Cevabı çıkarma fonksiyonu
def extract_answer(prediction):
    match = re.search(r"####\s*([\d.,]+)", prediction) # Daha genel bir regex
    if match:
        return match.group(1).strip().replace(",", "") # Virgülleri kaldır
    return None

In [23]:
# Test döngüsü (Sadece ARC için)
num_shots = 10
num_test_examples = 400
results = []

random.seed(42)
shot_indices = random.sample(range(len(dataset)), num_shots)
shot_examples = [dataset[i] for i in shot_indices]

# Seçilen shot örneklerini ve indekslerini sakla
with open("arc_shot_examples.json", "w", encoding="utf-8") as f:
    json.dump({"indices": shot_indices, "examples": shot_examples}, f, ensure_ascii=False, indent=4)

if len(dataset) < num_test_examples:
    num_test_examples = len(dataset)

for i in range(num_test_examples):
    example = dataset[i]
    prediction = get_prediction_arc(example, shot_examples)

    if 'answerKey_letter' in example: # Örnekte harfsel cevap varsa
        true_answer = example['answerKey_letter']
    else:
        try:
            true_answer_index = int(example["answerKey"]) - 1
            true_answer = chr(65 + true_answer_index)
        except ValueError: # Örnekte harfsel cevap varsa
            true_answer = example["answerKey"]
        except KeyError:
            true_answer = ""

    results.append({
        "question": example["question"],
        "true_answer": true_answer,
        "predicted_answer": prediction
    })
    if i % 20 == 0:
        print(f"ARC: {i}/{num_test_examples} örnek tamamlandı.")

# Sonuçları kaydet
with open("10_shot_results_arc.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("ARC 10-shot sonuçları kaydedildi.")

ARC: 0/400 örnek tamamlandı.
ARC: 20/400 örnek tamamlandı.


KeyboardInterrupt: 

In [33]:
def calculate_accuracy_arc(results_file):
    with open(results_file, "r", encoding="utf-8") as f:
        results = json.load(f)

    correct_predictions = 0
    total_predictions = len(results)

    for result in results:
        if result["predicted_answer"].strip() == result["true_answer"]:  # Tahmin ve gerçek cevapları karşılaştır
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

# Sonuç dosyanızın adını buraya girin
results_file = "10_shot_results_arc.json"  # Dosya adınızı doğru girin

accuracy = calculate_accuracy_arc(results_file)
print(f"ARC 10-shot Başarısı: {accuracy * 100:.2f}%")

ARC 10-shot Başarısı: 26.25%


In [32]:
# Test döngüsü (Sadece Hellaswag için)
num_shots = 10
num_test_examples = 400
results = []

random.seed(42)  # Sabit seed
shot_indices = random.sample(range(len(dataset_hellas)), num_shots)
shot_examples = [dataset_hellas[i] for i in shot_indices]

# Seçilen shot örneklerini ve indekslerini kaydet
with open("hellaswag_shot_examples.json", "w", encoding="utf-8") as f:
    json.dump({"indices": shot_indices, "examples": shot_examples}, f, ensure_ascii=False, indent=4)

if len(dataset_hellas) < num_test_examples:
    num_test_examples = len(dataset_hellas)

for i in range(num_test_examples):
    example = dataset_hellas[i]
    prediction = get_prediction_hellaswag(example, shot_examples)
    true_answer = example['label']

    results.append({
        "ctx": example["ctx"], # Doğrudan ctx'i kullan
        "true_answer": true_answer,
        "predicted_answer": prediction,
        "shot_indices": shot_indices
    })
    if i % 20 == 0:
        print(f"Hellaswag: {i}/{num_test_examples} örnek tamamlandı.")

# Sonuçları kaydet (shot indekslerini de içerir)
with open("10_shot_results_hellaswag.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Hellaswag 10-shot sonuçları kaydedildi.")

Hellaswag: 0/400 örnek tamamlandı.
Hellaswag: 20/400 örnek tamamlandı.
Hellaswag: 40/400 örnek tamamlandı.
Hellaswag: 60/400 örnek tamamlandı.
Hellaswag: 80/400 örnek tamamlandı.
Hellaswag: 100/400 örnek tamamlandı.
Hellaswag: 120/400 örnek tamamlandı.
Hellaswag: 140/400 örnek tamamlandı.
Hellaswag: 160/400 örnek tamamlandı.
Hellaswag: 180/400 örnek tamamlandı.
Hellaswag: 200/400 örnek tamamlandı.
Hellaswag: 220/400 örnek tamamlandı.
Hellaswag: 240/400 örnek tamamlandı.
Hellaswag: 260/400 örnek tamamlandı.
Hellaswag: 280/400 örnek tamamlandı.
Hellaswag: 300/400 örnek tamamlandı.
Hellaswag: 320/400 örnek tamamlandı.
Hellaswag: 340/400 örnek tamamlandı.
Hellaswag: 360/400 örnek tamamlandı.
Hellaswag: 380/400 örnek tamamlandı.
Hellaswag 10-shot sonuçları kaydedildi.


In [41]:
# Test döngüsü (GMSK8K için)
num_shots = 10
num_test_examples = 400
results = []

random.seed(42)
shot_indices = random.sample(range(len(dataset_gsm)), num_shots)
shot_examples = [dataset_gsm[i] for i in shot_indices]

with open("gsm8k_shot_examples.json", "w", encoding="utf-8") as f:
    json.dump({"indices": shot_indices, "examples": shot_examples}, f, ensure_ascii=False, indent=4)

if len(dataset_gsm) < num_test_examples:
    num_test_examples = len(dataset_gsm)

for i in range(num_test_examples):
    example = dataset_gsm[i]
    prediction = get_prediction_gsm(example, shot_examples)
    extracted_answer = extract_answer(prediction)
    true_answer = extract_answer(example['answer'])

    results.append({
        "question": example["question"],
        "true_answer": true_answer,
        "predicted_answer": extracted_answer,
        "shot_indices": shot_indices
    })
    if i % 20 == 0:
        print(f"GMSK8K: {i}/{num_test_examples} örnek tamamlandı.")

with open("10_shot_results_gsm8k.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("GMSK8K 10-shot sonuçları kaydedildi.")


GMSK8K: 0/400 örnek tamamlandı.
GMSK8K: 20/400 örnek tamamlandı.
GMSK8K: 40/400 örnek tamamlandı.
GMSK8K: 60/400 örnek tamamlandı.
GMSK8K: 80/400 örnek tamamlandı.
GMSK8K: 100/400 örnek tamamlandı.
GMSK8K: 120/400 örnek tamamlandı.
GMSK8K: 140/400 örnek tamamlandı.
GMSK8K: 160/400 örnek tamamlandı.
GMSK8K: 180/400 örnek tamamlandı.
GMSK8K: 200/400 örnek tamamlandı.
GMSK8K: 220/400 örnek tamamlandı.
GMSK8K: 240/400 örnek tamamlandı.
GMSK8K: 260/400 örnek tamamlandı.
GMSK8K: 280/400 örnek tamamlandı.
GMSK8K: 300/400 örnek tamamlandı.
GMSK8K: 320/400 örnek tamamlandı.
GMSK8K: 340/400 örnek tamamlandı.
GMSK8K: 360/400 örnek tamamlandı.
GMSK8K: 380/400 örnek tamamlandı.
GMSK8K 10-shot sonuçları kaydedildi.


In [42]:
# Başarı hesaplama fonksiyonu (GMSK8K için)
def calculate_accuracy_gsm(results_file):
    with open(results_file, "r", encoding="utf-8") as f:
        results = json.load(f)

    correct_predictions = 0
    total_predictions = len(results)

    for result in results:
      if result["predicted_answer"] is not None and result["true_answer"] is not None: #None kontrolü eklendi
        try:
          if float(result["predicted_answer"]) == float(result["true_answer"]): # float karşılaştırması
            correct_predictions += 1
        except ValueError:
            print(f"ValueError: Predicted: {result['predicted_answer']}, True: {result['true_answer']}")

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

accuracy = calculate_accuracy_gsm("10_shot_results_gsm8k.json")
print(f"GMSK8K 10-shot Başarısı: {accuracy * 100:.2f}%")

GMSK8K 10-shot Başarısı: 23.00%
